In [1]:
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate

In [2]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')

In [30]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')

up = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                INSERT DATA
                { GRAPH  <https://bibliokeia.com/resources/work/bk-8> { 
                     <https://bibliokeia.com/resources/work/bk-8>
                      bf:title "TESTET"} }"""
response = acervoUpdate.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [31]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
up = """
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
DELETE WHERE {
  GRAPH <https://bibliokeia.com/resources/work/bk-8> {
     <https://bibliokeia.com/resources/work/bk-8>
    bf:title ?o}
  }
  INSERT DATA
                { GRAPH  <https://bibliokeia.com/resources/work/bk-8> { 
                     <https://bibliokeia.com/resources/work/bk-8>
                      bf:title "TESTE222"} }
"""
response = acervoUpdate.run_sparql(up)
response.convert()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Encountered " <INSERT_DATA> "INSERT DATA "" at line 8, column 3.\r\nWas expecting one of:\r\n    <EOF> \r\n    ";" ...\r\n    ";" ...\r\n    \n'

In [6]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
up = """
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
WITH <https://bibliokeia.com/resources/work/bk-36> 
DELETE { <https://bibliokeia.com/resources/work/bk-36> bf:title ?o .
            ?s ?p ?o } 
WHERE { <https://bibliokeia.com/resources/work/bk-36> bf:title ?o .
            ?s ?p ?o }
"""
response = acervoUpdate.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Title

In [18]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')

up = """
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

WITH <https://bibliokeia.com/resources/work/bk-22>
DELETE
{
   <https://bibliokeia.com/resources/work/bk-22> bf:title ?o .
  	?o rdfs:label ?title .
}
INSERT
{
   <https://bibliokeia.com/resources/work/bk-22> bf:title ?o .
  	?o rdfs:label "TESTE1" .
}
WHERE { 
<https://bibliokeia.com/resources/work/bk-22> bf:title ?o .
  	?o rdfs:label ?title .
    }
"""

response = acervoUpdate.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [22]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')

up = """
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

WITH <https://bibliokeia.com/resources/work/bk-7>
DELETE
{
   <https://bibliokeia.com/resources/work/bk-7> bf:title ?o .
  	?o ?p ?title .
}
INSERT
{
   <https://bibliokeia.com/resources/work/bk-7> bf:title ?o .
  	?o rdfs:label "TESTE4" .

    <https://bibliokeia.com/resources/work/bk-7> bf:title ?o .
  	?o bf:mainTitle "mainTitle1" .
}
WHERE { 
<https://bibliokeia.com/resources/work/bk-7> bf:title ?o .
  	?o ?p ?title .
    }
"""

response = acervoUpdate.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
def Title()